# LangChain 核心模块学习：Chains

对于简单的大模型应用，单独使用语言模型（LLMs）是可以的。

**但更复杂的大模型应用需要将 `LLMs` 和 `Chat Models` 链接在一起 - 要么彼此链接，要么与其他组件链接。**

LangChain 为这种“链式”应用程序提供了 `Chain` 接口。

LangChain 以通用方式定义了 `Chain`，它是对组件进行调用序列的集合，其中可以包含其他链。

In [1]:
# 去掉pip install 使用项目依赖版本
# ! pip install -U langchain

## Chain Class 基类

类继承关系：

```
Chain --> <name>Chain  # Examples: LLMChain, MapReduceChain, RouterChain
```

**代码实现：https://github.com/langchain-ai/langchain/blob/master/libs/langchain/langchain/chains/base.py**

```python
# 定义一个名为Chain的基础类
class Chain(Serializable, Runnable[Dict[str, Any], Dict[str, Any]], ABC):
    """为创建结构化的组件调用序列的抽象基类。
    
    链应该用来编码对组件的一系列调用，如模型、文档检索器、其他链等，并为此序列提供一个简单的接口。
    
    Chain接口使创建应用程序变得容易，这些应用程序是：
    - 有状态的：给任何Chain添加Memory可以使它具有状态，
    - 可观察的：向Chain传递Callbacks来执行额外的功能，如记录，这在主要的组件调用序列之外，
    - 可组合的：Chain API足够灵活，可以轻松地将Chains与其他组件结合起来，包括其他Chains。
    
    链公开的主要方法是：
    - `__call__`：链是可以调用的。`__call__`方法是执行Chain的主要方式。它将输入作为一个字典接收，并返回一个字典输出。
    - `run`：一个方便的方法，它以args/kwargs的形式接收输入，并将输出作为字符串或对象返回。这种方法只能用于一部分链，不能像`__call__`那样返回丰富的输出。
    """

    # 调用链
    def invoke(
        self, input: Dict[str, Any], config: Optional[runnableConfig] = None
    ) -> Dict[str, Any]:
        """传统调用方法。"""
        return self(input, **(config or {}))

    # 链的记忆，保存状态和变量
    memory: Optional[BaseMemory] = None
    """可选的内存对象，默认为None。
    内存是一个在每个链的开始和结束时被调用的类。在开始时，内存加载变量并在链中传递它们。在结束时，它保存任何返回的变量。
    有许多不同类型的内存，请查看内存文档以获取完整的目录。"""

    # 回调，可能用于链的某些操作或事件。
    callbacks: Callbacks = Field(default=None, exclude=True)
    """可选的回调处理程序列表（或回调管理器）。默认为None。
    在对链的调用的生命周期中，从on_chain_start开始，到on_chain_end或on_chain_error结束，都会调用回调处理程序。
    每个自定义链可以选择调用额外的回调方法，详细信息请参见Callback文档。"""

    # 是否详细输出模式
    verbose: bool = Field(default_factory=_get_verbosity)
    """是否以详细模式运行。在详细模式下，一些中间日志将打印到控制台。默认值为`langchain.verbose`。"""

    # 与链关联的标签
    tags: Optional[List[str]] = None
    """与链关联的可选标签列表，默认为None。
    这些标签将与对这个链的每次调用关联起来，并作为参数传递给在`callbacks`中定义的处理程序。
    你可以使用这些来例如识别链的特定实例与其用例。"""

    # 与链关联的元数据
    metadata: Optional[Dict[str, Any]] = None
    """与链关联的可选元数据，默认为None。
    这些元数据将与对这个链的每次调用关联起来，并作为参数传递给在`callbacks`中定义的处理程序。
    你可以使用这些来例如识别链的特定实例与其用例。"""
```

## LLMChain

LLMChain 是 LangChain 中最简单的链，作为其他复杂 Chains 和 Agents 的内部调用，被广泛应用。

一个LLMChain由PromptTemplate和语言模型（LLM or Chat Model）组成。它使用直接传入（或 memory 提供）的 key-value 来规范化生成 Prompt Template（提示模板），并将生成的 prompt （格式化后的字符串）传递给大模型，并返回大模型输出。

![](../images/llm_chain.png)

In [3]:
# from langchain_openai import OpenAI
# from langchain.prompts import PromptTemplate

# llm = OpenAI(model_name="gpt-3.5-turbo-instruct", temperature=0.9, max_tokens=500)

In [1]:
# 【新增】ChatOpenAI调用方式，使用gpt-4o-mini模型
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate

llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0.9, max_tokens=500)

In [2]:
prompt = PromptTemplate(
    input_variables=["product"],
    template="给制造{product}的有限公司取10个好名字，并给出完整的公司名称",
)

In [5]:
from langchain.chains import LLMChain

chain = LLMChain(llm=llm, prompt=prompt)
# chain = prompt | llm
print(chain.invoke({
    'product': "性能卓越的GPU"
    }))

C:\Users\huangaf\AppData\Local\Temp\ipykernel_8064\4178063474.py:3: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt)


{'product': '性能卓越的GPU', 'text': '当然，以下是10个适合制造性能卓越GPU公司的名称及其完整公司名称：\n\n1. **超速图形科技有限公司**\n   - UltraSpeed Graphics Technology Co., Ltd.\n\n2. **极致视觉处理股份公司**\n   - Extreme Vision Processing Corporation\n\n3. **未来图像动力有限公司**\n   - Future Image Dynamics Co., Ltd.\n\n4. **强效图形创新科技公司**\n   - PowerGraphics Innovation Technology Corp.\n\n5. **顶尖图形解决方案有限公司**\n   - Top Tier Graphics Solutions Co., Ltd.\n\n6. **先锋GPU设计公司**\n   - Pioneer GPU Design Co.\n\n7. **显像天才科技有限公司**\n   - Imaging Genius Technology Co., Ltd.\n\n8. **超越性能显卡有限公司**\n   - Beyond Performance GPUs Co., Ltd.\n\n9. **精英图形硬件有限公司**\n   - Elite Graphics Hardware Co., Ltd.\n\n10. **高效能显卡研发公司**\n    - High Performance Graphics R&D Corp.\n\n希望这些名字能够为您提供灵感！'}


In [ ]:
# 详细输出模式
chain.verbose =True

In [7]:
chain.verbose

True

In [8]:
print(chain.invoke({
    'product': "性能卓越的GPU"
    }))



> Entering new LLMChain chain...
Prompt after formatting:
给制造性能卓越的GPU的有限公司取10个好名字，并给出完整的公司名称

> Finished chain.
{'product': '性能卓越的GPU', 'text': '当然可以！以下是10个制造性能卓越的GPU公司的名字及其完整名称：\n\n1. **超锐图形科技有限公司**  \n   (SuperSharp Graphics Technology Co., Ltd.)\n\n2. **极致性能视觉有限公司**  \n   (Ultimate Performance Visuals Co., Ltd.)\n\n3. **未来图形解决方案有限公司**  \n   (Future Graphics Solutions Co., Ltd.)\n\n4. **精英图像处理有限公司**  \n   (Elite Image Processing Co., Ltd.)\n\n5. **超能显卡开发有限公司**  \n   (SuperPower GPU Development Co., Ltd.)\n\n6. **顶级图形引擎有限公司**  \n   (TopTier Graphics Engine Co., Ltd.)\n\n7. **创新视觉计算有限公司**  \n   (Innovative Visual Computing Co., Ltd.)\n\n8. **领航GPU技术有限公司**  \n   (Pioneering GPU Technology Co., Ltd.)\n\n9. **高效图形硬件有限公司**  \n   (Efficient Graphics Hardware Co., Ltd.)\n\n10. **专业图形驱动有限公司**  \n    (Professional Graphics Drivers Co., Ltd.)\n\n希望这些名字能够激发您的灵感！'}


## Sequential Chain

串联式调用语言模型（将一个调用的输出作为另一个调用的输入）。

顺序链（Sequential Chain ）允许用户连接多个链并将它们组合成执行特定场景的流水线（Pipeline）。有两种类型的顺序链：

- SimpleSequentialChain：最简单形式的顺序链，每个步骤都具有单一输入/输出，并且一个步骤的输出是下一个步骤的输入。
- SequentialChain：更通用形式的顺序链，允许多个输入/输出。

### 使用 SimpleSequentialChain 实现戏剧摘要和评论（单输入/单输出）

![](../images/simple_sequential_chain_0.png)

In [9]:
# 这是一个 LLMChain，用于根据剧目的标题撰写简介。
# llm = OpenAI(temperature=0.7, max_tokens=1000)

# 【新增】ChatOpenAI调用方式，使用gpt-4o-mini模型
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0.9, max_tokens=500)

template = """你是一位剧作家。根据戏剧的标题，你的任务是为该标题写一个简介。

标题：{title}
剧作家：以下是对上述戏剧的简介："""

prompt_template = PromptTemplate(input_variables=["title"], template=template)
synopsis_chain = LLMChain(llm=llm, prompt=prompt_template)

In [10]:
# 这是一个LLMChain，用于根据剧情简介撰写一篇戏剧评论。
# llm = OpenAI(temperature=0.7, max_tokens=1000)
template = """你是《纽约时报》的戏剧评论家。根据剧情简介，你的工作是为该剧撰写一篇评论。

剧情简介：
{synopsis}

以下是来自《纽约时报》戏剧评论家对上述剧目的评论："""

prompt_template = PromptTemplate(input_variables=["synopsis"], template=template)
review_chain = LLMChain(llm=llm, prompt=prompt_template)

![](../images/simple_sequential_chain_1.png)

In [11]:
# 这是一个SimpleSequentialChain，按顺序运行这两个链
from langchain.chains import SimpleSequentialChain

overall_chain = SimpleSequentialChain(chains=[synopsis_chain, review_chain], verbose=True)

In [12]:
review = overall_chain.invoke("三体人不是无法战胜的")



> Entering new SimpleSequentialChain chain...
**剧作简介：三体人不是无法战胜的**

在一个被外星文明“三体人”统治的未来，人类面临着前所未有的危机。这些高智商的外星人以极其先进的科技和冷酷的逻辑主导着地球，他们对人类的生存权利提出了严峻的挑战。在绝望的边缘，人类的抵抗看似微不足道。

然而，在这场漫长的斗争中，几位来自不同背景的普通人聚集在一起，亟待找寻打破绝境的希望。他们是科学家、工程师、士兵和普通市民，尽管背景不同，但都共同追求着一种信念：团结才能战胜一切。通过智慧与勇气，他们逐渐揭开了三体人的弱点，并利用创新的科技和人类的情感力量来反抗。

随着剧情的发展，观众将见证人类从迷茫、失落到崛起反抗的演变过程。戏剧不仅探讨了科技与伦理的碰撞，也深刻反映了人类在面对强大敌人时的坚韧与团结。最终，剧中的角色们意识到，真正的胜利并不仅仅是战胜敌人，更在于发现自我、信任彼此的力量。

《三体人不是无法战胜的》是一部关于希望、抗争与人性的戏剧，激励观众在绝境中寻找光明，展现出人类在逆境中如何焕发出无限潜力。
**评论：希望之光在绝境中闪耀——《三体人不是无法战胜的》**

在当今这个科技高速发展的时代，很多戏剧试图探讨人类与科技的关系。而《三体人不是无法战胜的》以其独特的视角和深邃的主题，将观众带入了一个令人不安的未来——一个被外星文明统治的地球。尽管这一设定的来源可以追溯到刘慈欣的原著，但剧作在舞台上的再现却赋予了这一故事全新的生命。

剧中，三体人的统治如同一把高悬的利剑，给人类带来了深重的绝望。然而，在这绝望的边缘，几位来自不同背景的普通人团结在一起，向着希望进发。这一充满人性光辉的故事深刻探讨了团结与信任的力量。正是这种力量，让他们能够在绝境中寻找突破口，也让观众感受到一种强烈的共鸣。

剧作的结构和节奏恰到好处，通过普通角色的多元背景，编织出一幅丰富而立体的人性画卷。无论是科学家、士兵还是普通市民，每个人都在阴暗处绽放出自己的光彩，展现出人类面对困境时的勇气与创造力。导演巧妙地运用舞台效果，将先进科技与人类情感相结合，呈现出既震撼又感人的场景，时常让人不由自主地屏住呼吸。

值得一提的是，剧本中对科技与伦理的碰撞进行了深刻的剖析。在面对三体人这种高度理性的存在时，剧中的人物不仅要争取生存，更要质疑如何在科技的洪流

In [13]:
review = overall_chain.invoke("星球大战第九季")



> Entering new SimpleSequentialChain chain...
《星球大战第九季》是一部恢弘壮阔的科幻剧作，延续了前八季的传奇故事，带领观众踏入一个充满了宇宙冒险、英雄成长与道德抉择的广阔星际世界。在这一季中，银河系面临着前所未有的威胁，黑暗势力悄然崛起，平衡宇宙秩序的希望寄托在几位年轻的绝地和勇敢的反抗军战士身上。

故事围绕主人公艾莉娜展开，她是一名年轻的绝地学徒，继承了强大的力量和责任。她与她的伙伴们必须穿越多个星球，面对复杂的敌对势力，探索古老的绝地遗迹，揭开隐藏在宇宙深处的秘密。在这一过程中，艾莉娜不仅要战胜外在的敌人，还要面对内心的挣扎与恐惧，寻找自己的身份与使命。

与此同时，经典角色如汉·索罗、Leia公主演与新一代角色的交汇，使得这一季充满了情感的碰撞与继承的意义。每个角色都在寻找自己的道路，面对亲情、友情与背叛，最终在星际纷争中发现真正的自我。

《星球大战第九季》不仅是一次视觉与听觉的盛宴，更是一部探讨勇气、牺牲与希望的深刻剧作，唤起观众对正义与团结的思考，激励每个人在自己的生活中追求光明与和平。
### 评论：星际传奇的延续——《星球大战第九季》

在“星球大战”这一传奇系列的第九季中，编剧和导演成功地将我们带入一个充满奇幻与危机的宇宙，既承载着对前八季的致敬，又勇敢地探索新的人物与故事。在这个时空交错的故事中，观众不仅仅是目睹了一场视觉与听觉的盛宴，更是在一段复杂情感的旅程中，对勇气与希望进行了深刻的思考。

核心角色艾莉娜的塑造尤为出色。作为新一代的绝地学徒，她的成长与挣扎反映了人们在现实生活中对身份与使命的探索。演员的表演自然而真挚，细腻地展现了艾莉娜从懵懂到坚定的过程。观众不仅能感受到她面对黑暗势力的恐惧，还能体会到她在道德抉择中的痛苦挣扎。这种情感的深度使得艾莉娜不仅仅是一个战士，更是一个立体的人物，她的故事引发了对年轻一代在历史洪流中角色的新思考。

经典角色汉·索罗和Leia公主的回归则是这一季的一大亮点。他们的存在不仅为老粉丝带来了情感的共鸣，更为新角色的成长提供了力量与智慧的指引。通过他们的互动，整部剧作展现了时间的流逝与责任的传承，强调了在纷争中保持团结的重要性。

剧中复杂的情节设定与视觉效果令观众目不暇接，宏大的星际战争场面令人肾上腺素飙升，而细腻的人际关系则给人以思考的空间。无论是壮丽

### 使用 SequentialChain 实现戏剧摘要和评论（多输入/多输出）

![](../images/sequential_chain_0.png)

In [14]:
# # 这是一个 LLMChain，根据剧名和设定的时代来撰写剧情简介。
# llm = OpenAI(temperature=.7, max_tokens=1000)

# # 【新增】ChatOpenAI调用方式，使用gpt-4o-mini模型
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0.9, max_tokens=500)

template = """你是一位剧作家。根据戏剧的标题和设定的时代，你的任务是为该标题写一个简介。

标题：{title}
时代：{era}
剧作家：以下是对上述戏剧的简介："""

prompt_template = PromptTemplate(input_variables=["title", "era"], template=template)
# output_key
synopsis_chain = LLMChain(llm=llm, prompt=prompt_template, output_key="synopsis", verbose=True)

In [15]:
# 这是一个LLMChain，用于根据剧情简介撰写一篇戏剧评论。

template = """你是《纽约时报》的戏剧评论家。根据该剧的剧情简介，你需要撰写一篇关于该剧的评论。

剧情简介：
{synopsis}

来自《纽约时报》戏剧评论家对上述剧目的评价："""

prompt_template = PromptTemplate(input_variables=["synopsis"], template=template)
review_chain = LLMChain(llm=llm, prompt=prompt_template, output_key="review", verbose=True)

In [16]:
from langchain.chains import SequentialChain

m_overall_chain = SequentialChain(
    chains=[synopsis_chain, review_chain],
    input_variables=["era", "title"],
    # Here we return multiple variables
    output_variables=["synopsis", "review"],
    verbose=True)

In [17]:
m_overall_chain.invoke({"title":"三体人不是无法战胜的", "era": "二十一世纪的新中国"})



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
你是一位剧作家。根据戏剧的标题和设定的时代，你的任务是为该标题写一个简介。

标题：三体人不是无法战胜的
时代：二十一世纪的新中国
剧作家：以下是对上述戏剧的简介：

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
你是《纽约时报》的戏剧评论家。根据该剧的剧情简介，你需要撰写一篇关于该剧的评论。

剧情简介：
**剧作简介：三体人不是无法战胜的**

在二十一世纪的新中国，科技飞速发展，人工智能与量子计算的结合推动了人类文明的进步。然而，外星文明的威胁悄然降临，三体人以其超乎想象的科技和智慧，试图侵占地球，改变人类的命运。

故事围绕着一群来自不同背景和学科的年轻科学家，他们在一次偶然的机会中发现了三体人技术的一个关键弱点。每个角色的个性和专业知识在面对外星敌人时各展所长，从物理学家到计算机专家，从生物学家到心理学家，他们在巨大的压力下展开了紧张的合作与斗争。

剧中不仅展现了科技与人性的碰撞，也探讨了人类在面对绝境时的团结与脆弱。随着剧情的发展，角色们经历了信任与背叛、牺牲与救赎，逐步认识到，战胜三体人并不仅仅依赖于科技的进步，更需要理解与包容，包括对自身的反思和对彼此的信任。

在经历了重重考验和智慧的碰撞后，最终他们找到了一个出人意料的解决方案，利用三体人对人类心理的误判，反转了战局。剧本结尾，主角们站在地球的阳光下，思考着人类未来的无限可能，展现了希望与团结的力量。

**《三体人不是无法战胜的》**不仅是一部科幻剧，更是对人类勇气与智慧的颂歌，提醒我们在面对强大敌人时，团结与创新才是我们最强的武器。

来自《纽约时报》戏剧评论家对上述剧目的评价：

> Finished chain.

> Finished chain.


{'title': '三体人不是无法战胜的',
 'era': '二十一世纪的新中国',
 'synopsis': '**剧作简介：三体人不是无法战胜的**\n\n在二十一世纪的新中国，科技飞速发展，人工智能与量子计算的结合推动了人类文明的进步。然而，外星文明的威胁悄然降临，三体人以其超乎想象的科技和智慧，试图侵占地球，改变人类的命运。\n\n故事围绕着一群来自不同背景和学科的年轻科学家，他们在一次偶然的机会中发现了三体人技术的一个关键弱点。每个角色的个性和专业知识在面对外星敌人时各展所长，从物理学家到计算机专家，从生物学家到心理学家，他们在巨大的压力下展开了紧张的合作与斗争。\n\n剧中不仅展现了科技与人性的碰撞，也探讨了人类在面对绝境时的团结与脆弱。随着剧情的发展，角色们经历了信任与背叛、牺牲与救赎，逐步认识到，战胜三体人并不仅仅依赖于科技的进步，更需要理解与包容，包括对自身的反思和对彼此的信任。\n\n在经历了重重考验和智慧的碰撞后，最终他们找到了一个出人意料的解决方案，利用三体人对人类心理的误判，反转了战局。剧本结尾，主角们站在地球的阳光下，思考着人类未来的无限可能，展现了希望与团结的力量。\n\n**《三体人不是无法战胜的》**不仅是一部科幻剧，更是对人类勇气与智慧的颂歌，提醒我们在面对强大敌人时，团结与创新才是我们最强的武器。',
 'review': '标题：《三体人不是无法战胜的》——科技与人性的交响曲\n\n在迅速发展的科技时代，《三体人不是无法战胜的》这部剧作无疑为观众们提供了一场引人深思的心灵之旅。通过将尖端科技与人类情感紧密结合，剧本不仅构建了一个激动人心的外星入侵故事，还深入探讨了人与人之间的信任与合作，展现了人类在绝境中的勇气与韧性。\n\n剧中的年轻科学家们来自不同的背景与专业，他们的个性鲜明，各自的知识与技能在抗击三体人这一共同目标下交融出火花。物理学家的理性分析、计算机专家的技术突破，生物学家的生态意识和心理学家的洞察力，共同构成了团队对抗外星威胁的基石。这种角色的多样化不仅增加了剧情的复杂性，也让观众看到了集体智慧的力量。\n\n尤其值得一提的是，剧中对科技与人性的深刻见解。随着外星文明的威胁日益加剧，科学家的团队不仅要依靠先进的技术，更需要相互理解与支持。在剧中，角色们经历了信任的建立与破裂，面对绝境时的团结与脆弱，让人不禁思考

In [21]:
#### The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
### 【新增】langchain 0.3版本，使用RunnableSequence替换LLMChain，并指定 output_key

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnableSequence

llm = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    temperature=0,
    max_tokens=200,
    verbose=True
)

summarizing_prompt_template = """
总结以下文本为一个 20 字以内的句子:
---
{content}
"""
summarizing_prompt = PromptTemplate.from_template(summarizing_prompt_template)
summarizing_chain = summarizing_prompt | llm | StrOutputParser()

translating_prompt_template = """
将{summary}翻译成英文:
"""
translating_prompt = PromptTemplate.from_template(translating_prompt_template)
translating_chain = translating_prompt | llm | StrOutputParser()

# Construct a RunnableSequence with custom output keys
overall_chain = summarizing_chain | {
    'summary': summarizing_chain,
    'translation': translating_chain
}

test_contetent = """
端到端在深度学习中指的是一种模型架构设计理念：
从原始输入数据到最终输出结果，整个决策过程完全由单一神经网络完成，无需人工设计中间处理环节。
这种设计摒弃了传统分步骤、模块化的处理流程，让模型自主挖掘数据中隐藏的复杂关联。
"""

result = overall_chain.invoke({"content": test_contetent})
print(result)

{'summary': '端到端深度学习模型自主挖掘数据关联，无需人工设计中间处理环节，完成整个决策过程。', 'translation': 'End-to-end deep learning models are completely handled by a single neural network to complete the entire decision-making process, autonomously uncovering complex correlations in the data without the need for manually designed intermediate processing steps.'}


### Homework

#### 使用 OutputParser 优化 overall_chain 输出格式，区分 synopsis_chain 和 review_chain 的结果

In [22]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnableSequence, RunnableLambda

# 设置 verbose 为 True
llm = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    temperature=0,
    max_tokens=200,
    verbose=True
)

summarizing_prompt_template = """
总结以下文本为一个 20 字以内的句子:
---
{content}
"""
summarizing_prompt = PromptTemplate.from_template(summarizing_prompt_template)
summarizing_chain = summarizing_prompt | llm | StrOutputParser()

translating_prompt_template = """
将以下文本翻译成英文:
{summary}
"""
translating_prompt = PromptTemplate.from_template(translating_prompt_template)

# 修改 translating_chain 的输入，使其接收 summary
translating_chain = {
    "summary": lambda x: x
} | translating_prompt | llm | StrOutputParser()

# 自定义输出解析器，将结果包装成字典
def custom_output_parser(summary_and_translation):
    summary = summary_and_translation['summary']
    translation = summary_and_translation['translation']
    return {
        'summary': summary,
        'translation': translation
    }

# 将自定义输出解析器包装成 RunnableLambda
output_parser_runnable = RunnableLambda(custom_output_parser)

# 构建 overall_chain 并使用自定义输出解析器
overall_chain = summarizing_chain | {
    'summary': lambda x: x,
    'translation': translating_chain
} | output_parser_runnable

test_contetent = """
端到端在深度学习中指的是一种模型架构设计理念：
从原始输入数据到最终输出结果，整个决策过程完全由单一神经网络完成，无需人工设计中间处理环节。
这种设计摒弃了传统分步骤、模块化的处理流程，让模型自主挖掘数据中隐藏的复杂关联。
"""

result = overall_chain.invoke({"content": test_contetent})
print(result)
    

{'summary': '端到端深度学习模型完全由单一神经网络完成整个决策过程，自主挖掘数据中的复杂关联。', 'translation': 'End-to-end deep learning models are entirely completed by a single neural network, autonomously uncovering complex correlations within the data.'}
